In [115]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import os
import numpy as np
from PIL import Image
import torch
import math
from skimage import filters
from scipy import ndimage
import random
import torch
import nibabel as nb
from glob import glob
from torchvision import transforms
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from trainer import run_training
from unetr import UNETR
from scipy.ndimage import zoom
import torch.nn.functional as F
import math
import torch.nn.init as init
import torch.nn as nn
from monai.inferers import sliding_window_inference
from monai.losses import DiceCELoss, DiceLoss
from monai.metrics import DiceMetric
from monai.transforms import Activations, AsDiscrete, Compose
from monai.utils.enums import MetricReduction
from functools import partial
from lr_scheduler import LinearWarmupCosineAnnealingLR
from argparse import Namespace
from torchvision.transforms import Compose
from torchvision.transforms import Normalize
import plotly.express as px 

In [16]:
args = Namespace(
    checkpoint=None,
    logdir="test",
    pretrained_dir="/home/sarvagya-pc/Desktop/Balgrist_neuroimg/UNETR/pretrained_model/",
    data_dir="/dataset/dataset0/",
    json_list="dataset_0.json",
    pretrained_model_name="UNETR_model_best_acc.pth",
    save_checkpoint=True,  # Set to True if you want to save checkpoints
    max_epochs=500,
    batch_size=1,
    sw_batch_size=1,
    optim_lr=1e-4,
    optim_name="adamw",
    reg_weight=1e-5,
    momentum=0.99,
    noamp=False,  # Set to True if you don't want AMP (Automatic Mixed Precision)
    val_every=10,
    distributed=False,  # Set to True for distributed training
    world_size=1,
    rank=0,
    dist_url="tcp://127.0.0.1:23456",
    dist_backend="nccl",
    workers=8,
    model_name="unetr",
    pos_embed="perceptron",
    norm_name="instance",
    num_heads=12,
    mlp_dim=3072,
    hidden_size=768,
    feature_size=16,
    in_channels=1,
    out_channels=1,
    res_block=False,  # Set to True if using residual blocks
    conv_block=False,  # Set to True if using convolutional blocks
    use_normal_dataset=False,  # Set to True if using MONAI's Dataset class
    a_min=-175.0,
    a_max=250.0,
    b_min=0.0,
    b_max=1.0,
    space_x=1.5,
    space_y=1.5,
    space_z=2.0,
    roi_x=96,
    roi_y=96,
    roi_z=3,
    dropout_rate=0.0,
    RandFlipd_prob=0.2,
    RandRotate90d_prob=0.2,
    RandScaleIntensityd_prob=0.1,
    RandShiftIntensityd_prob=0.1,
    infer_overlap=0.5,
    lrschedule="warmup_cosine",
    warmup_epochs=50,
    resume_ckpt=False,
    resume_jit=False,
    smooth_dr=1e-6,
    smooth_nr=0.0
)

args.amp = not args.noamp

In [18]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [19]:
folder = "/media/sarvagya-pc/2TB HDD/Balgrist/GM_mask/new_data/"
subs = sorted(os.listdir(folder))
# ids = ids[6:]
# ids.remove("sub-9043")
# to_remove = ['sub-01', 'sub-06','sub-73', 'sub-74', 'sub-75', 'sub-76', 'sub-77']
# for item in to_remove:
#     if item in subs:
#         subs.remove(item)
# subs.pop(0)
# subs.pop(0)
# subs.pop(0)
print(subs)

image_nifti = []
label_nifti = []
for sub in subs:
    sessions = sorted([ses for ses in os.listdir(folder+sub+'/') if "ses-" in ses])
    for ses in sessions:
        file_image = glob(folder+sub+'/'+ses+'/anat/*_acq-lumbarMEGRE3D_desc-crop_T2starw.nii')
        file_label = glob(folder+sub+'/'+ses+'/anat/*_acq-lumbarMEGRE3D_desc-crop_seg-manual_label-SC_mask.nii')
        
        if len(file_label)==0:
            print("this isbject is empty "+sub)
            pass
        else:
            print(file_image[0])
            # print(file)
            print(file_image[0].split('/')[-1].split("_")[0])
            if file_image[0].split('/')[-1].split("_")[0] != sub:
                print("error with "+sub)
            else:
                image_nifti.append(file_image[0])
                label_nifti.append(file_label[0])

X_train, X_test, y_train, y_test = train_test_split(image_nifti, label_nifti, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

['sub-01', 'sub-02', 'sub-03', 'sub-04', 'sub-05', 'sub-06', 'sub-07', 'sub-08', 'sub-09', 'sub-10', 'sub-11', 'sub-12', 'sub-13', 'sub-14', 'sub-15', 'sub-16', 'sub-17', 'sub-18', 'sub-19', 'sub-20', 'sub-21', 'sub-22', 'sub-23', 'sub-24', 'sub-25', 'sub-26', 'sub-27', 'sub-28', 'sub-29', 'sub-30', 'sub-31', 'sub-32', 'sub-33', 'sub-34', 'sub-35', 'sub-36', 'sub-37', 'sub-38', 'sub-39', 'sub-40', 'sub-41', 'sub-42', 'sub-43', 'sub-44', 'sub-45', 'sub-46', 'sub-47', 'sub-48', 'sub-49', 'sub-50', 'sub-51', 'sub-52', 'sub-53', 'sub-54', 'sub-55', 'sub-56', 'sub-57', 'sub-58', 'sub-59', 'sub-60', 'sub-61', 'sub-62', 'sub-63', 'sub-64', 'sub-65', 'sub-66', 'sub-67', 'sub-68', 'sub-69', 'sub-70', 'sub-71', 'sub-72', 'sub-73', 'sub-74', 'sub-75', 'sub-76']
/media/sarvagya-pc/2TB HDD/Balgrist/GM_mask/new_data/sub-01/ses-01/anat/sub-01_ses-01_acq-lumbarMEGRE3D_desc-crop_T2starw.nii
sub-01
/media/sarvagya-pc/2TB HDD/Balgrist/GM_mask/new_data/sub-02/ses-01/anat/sub-02_ses-01_acq-lumbarMEGRE3D_de

In [12]:
class SpinalCordDataset(Dataset):
    def __init__(self, image_paths, label_paths, transform=None):
        assert len(image_paths) == len(label_paths), "Image and label paths must have the same length"
        self.image_paths = image_paths
        self.label_paths = label_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label_path = self.label_paths[idx]

        # Load the NIfTI files
        image = nb.load(image_path).get_fdata(dtype=np.float32)  # Shape: (H, W, D)
        label = nb.load(label_path).get_fdata(dtype=np.float32)  # Shape: (H, W, D)

        # Extract specific slices
        image_slices = image[:, :, :20]  # Extract slices 0-19
        label_slices = label[:, :, :20]  # Extract corresponding slices

        # Apply transforms to each slice
        transformed_images = []
        transformed_labels = []

        for i in range(image_slices.shape[2]):
            img_slice = image_slices[:, :, i]
            lbl_slice = label_slices[:, :, i]

            if self.transform:
                img_slice, lbl_slice = self.transform(img_slice, lbl_slice)

            # Convert to PyTorch tensors
            img_tensor = torch.tensor(img_slice, dtype=torch.float32).permute(2, 0, 1)  # Shape: (3, H, W)
            lbl_tensor = torch.tensor(lbl_slice, dtype=torch.float32).unsqueeze(0)      # Shape: (1, H, W)

            transformed_images.append(img_tensor)
            transformed_labels.append(lbl_tensor)

        # Stack slices to form 3D volumes
        transformed_images = torch.stack(transformed_images, dim=3)  # Shape: (20, 3, H, W)
        transformed_labels = torch.stack(transformed_labels, dim=3)  # Shape: (20, 1, H, W)

        

        return {
            "image": transformed_images,
            "label": transformed_labels
        }


In [13]:
class SpinalCordTransform:
    def __init__(self, target_size=(96, 96), flip_prob=0.5):
        self.target_size = target_size
        self.flip_prob = flip_prob

    def resize(self, image, label):
        """
        Resize image and label to the target size.
        """
        zoom_factors_image = (
            self.target_size[0] / image.shape[0],
            self.target_size[1] / image.shape[1]
        )
        zoom_factors_label = (
            self.target_size[0] / label.shape[0],
            self.target_size[1] / label.shape[1]
        )
        resized_image = zoom(image, zoom_factors_image, order=1)  # Bilinear interpolation
        resized_label = zoom(label, zoom_factors_label, order=0)  # Nearest-neighbor interpolation for labels
        return resized_image, resized_label

    def normalize(self, slice_2d):
        """
        Normalize intensity values to [0, 1].
        """
        slice_2d = (slice_2d - np.min(slice_2d)) / (np.max(slice_2d) - np.min(slice_2d) + 1e-8)
        return slice_2d

    def random_flip(self, image, label):
        """
        Randomly flip along spatial axes and ensure contiguous memory.
        """
        if np.random.rand() < self.flip_prob:
            image = np.flip(image, axis=0).copy()
            label = np.flip(label, axis=0).copy()
        if np.random.rand() < self.flip_prob:
            image = np.flip(image, axis=1).copy()
            label = np.flip(label, axis=1).copy()
        return image, label

    def grayscale_to_rgb(self, slice_2d):
        """
        Convert a grayscale slice to RGB by duplicating the grayscale values across three channels.
        """
        return np.stack([slice_2d] * 3, axis=-1)  # Shape: (H, W, 3)

    def __call__(self, image, label):
        """
        Apply all transformations to an image and label.
        """
        # Resize to target size
        image, label = self.resize(image, label)

        # Normalize image
        image = self.normalize(image)
        label = self.normalize(label)

        # Random flips
        image, label = self.random_flip(image, label)

        # Convert image to RGB
        # image = self.grayscale_to_rgb(image)  # Shape: (H, W, 3)
        image = image.reshape(image.shape[0], image.shape[1],1)

        return image, label


In [22]:
image_paths_train = X_train
label_paths_train = y_train

image_paths_val = X_val
label_paths_val = y_val

# # Create Dataset and DataLoader
# transform = SpinalCordTransform(flip_prob=0.5)
# dataset = SpinalCordDataset(image_paths=image_paths, label_paths=label_paths, transform=transform)

# data_loader = DataLoader(dataset, batch_size=2, shuffle=True, num_workers=4)

# Create instances of SpinalCordTransform with desired parameters
train_transform = SpinalCordTransform(target_size=(96, 96), flip_prob=0.5)
val_transform = SpinalCordTransform(target_size=(96, 96), flip_prob=0.0)  # No flipping for validation

# Datasets
train_dataset = SpinalCordDataset(image_paths=image_paths_train, label_paths=label_paths_train, transform=train_transform)
val_dataset = SpinalCordDataset(image_paths=image_paths_val, label_paths=label_paths_val, transform=val_transform)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False, num_workers=4)

# train_loader = train_loader.permute(0, 2, 3, 4, 1)
# val_loader = val_loader.permute(0, 2, 3, 4, 1)


In [23]:
for batch in train_loader:
    images = batch["image"]  # Shape: (batch_size, 20, 3, 96, 96)
    labels = batch["label"]  # Shape: (batch_size, 20, 1, 96, 96)
    print(f"Image batch shape: {images.shape}")
    print(f"Label batch shape: {labels.shape}")
    break

Image batch shape: torch.Size([2, 1, 96, 96, 20])
Label batch shape: torch.Size([2, 1, 96, 96, 20])


In [24]:
for batch in val_loader:
    images = batch["image"]  # Shape: (batch_size, 20, 3, 96, 96)
    labels = batch["label"]  # Shape: (batch_size, 20, 1, 96, 96)
    print(f"Image batch shape: {images.shape}")
    print(f"Label batch shape: {labels.shape}")
    break

Image batch shape: torch.Size([2, 1, 96, 96, 20])
Label batch shape: torch.Size([2, 1, 96, 96, 20])


In [25]:
batch = next(iter(val_loader))  # Get a single batch
images = batch["image"]  # Shape: (batch_size, channels, depth, height, width)
labels = batch["label"]  # Shape: (batch_size, channels, depth, height, width)

In [26]:
slice_idx = 10  # Choose an index from the depth axis
image_slice = images[0, 0, :, :, slice_idx]  # [batch, channel, depth, H, W]
label_slice = labels[0, 0, :, :, slice_idx]

In [27]:
# num_slices = images.shape[4]  # Number of slices in the depth dimension
# for i in range(num_slices):
#     plt.figure(figsize=(5, 5))
#     plt.imshow(images[0, 0, :, :, i].cpu().numpy(), cmap="gray", alpha=0.8)
#     plt.imshow(labels[0, 0, :, :, i].cpu().numpy(), cmap="jet", alpha=0.4)  # Overlay in color
#     plt.title("Image with Overlayed Label")
#     plt.axis("off")
#     plt.show()

In [28]:
# 1. Define the model
model = UNETR(
    in_channels=1,  # Grayscale MRI
    out_channels=1,  # Segmentation classes
    img_size=(96, 96, 20),  # Input size
    feature_size=16,
    hidden_size=768,
    mlp_dim=3072,
    num_heads=12,
    pos_embed="perceptron",
    norm_name="instance",
    conv_block=True,
    res_block=True,
    dropout_rate=0.0,
)

# 2. Load pre-trained weights
pretrained_weights = torch.load(
    "/home/sarvagya-pc/Desktop/Balgrist_neuroimg/UNETR/pretrained_model/UNETR_model_best_acc.pth",
    weights_only=True,
)

# 3. Edit the pre-trained weights


# Completely remove positional embeddings from pre-trained weights
if "vit.patch_embedding.position_embeddings" in pretrained_weights:
    del pretrained_weights["vit.patch_embedding.position_embeddings"]

# Remove the output layer weights
if "out.conv.conv.weight" in pretrained_weights:
    del pretrained_weights["out.conv.conv.weight"]
if "out.conv.conv.bias" in pretrained_weights:
    del pretrained_weights["out.conv.conv.bias"]

# Remove the patch embedding weights (if mismatch)
if "vit.patch_embedding.patch_embeddings.1.weight" in pretrained_weights:
    del pretrained_weights["vit.patch_embedding.patch_embeddings.1.weight"]

# 4. Load the edited weights into the model
model.load_state_dict(pretrained_weights, strict=False)

# 5. Reinitialize any layers that were removed or not loaded

# Completely reinitialize the positional embeddings
# num_patches = (96 // 16) * (96 // 16) * (1 // 1) # Number of patches

patch_size = (16, 16, 1)  # Match your depth dimension
input_size = (96, 96, 20)  # Input dimensions

# Calculate number of patches
num_patches = (input_size[0] // patch_size[0]) * \
              (input_size[1] // patch_size[1]) * \
              (input_size[2] // patch_size[2])  # 6 * 6 * 20 = 720

embedding_dim = model.vit.patch_embedding.patch_embeddings[1].out_features  # Hidden size

model.vit.patch_embedding.position_embeddings = nn.Parameter(
    torch.zeros(1, num_patches, embedding_dim)
)

torch.nn.init.trunc_normal_(model.vit.patch_embedding.position_embeddings, std=0.02)

# Reinitialize output layer
model.out.conv.conv.weight.data.normal_(0, 0.02)
model.out.conv.conv.bias.data.zero_()

model.to(device)

print("Pre-trained weights loaded and adjusted successfully!")

Pre-trained weights loaded and adjusted successfully!


In [29]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
dice_loss = DiceCELoss(to_onehot_y=False, sigmoid=True, squared_pred=True, smooth_nr=0.0, smooth_dr=1e-6)
dice_acc = DiceMetric(include_background=True, reduction=MetricReduction.MEAN, get_not_nans=True)
model_inferer = partial(
    sliding_window_inference,
    roi_size=[96,96,20],
    sw_batch_size=1,
    predictor=model,
    overlap=0.5,
)
scheduler = LinearWarmupCosineAnnealingLR(
            optimizer, warmup_epochs=50, max_epochs=5000
        )
start_epoch = 0
# post_label = AsDiscrete(to_onehot=False, n_classes=args.out_channels)
# post_pred = AsDiscrete(argmax=True, to_onehot=False, n_classes=args.out_channels)

# Ground truth labels: no transformation needed
post_label = AsDiscrete()  # Use as-is if labels are already binary

# Model predictions: threshold to binarize probabilities
post_pred = AsDiscrete(threshold_values=0.5)


In [30]:
accuracy = run_training(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    loss_func=dice_loss,
    args=args,
    acc_func=dice_acc,
    model_inferer=model_inferer,
    scheduler=scheduler,
    start_epoch=start_epoch,
    post_label=post_label,
    post_pred=post_pred
)

Writing Tensorboard logs to  test
0 Mon Dec  2 19:50:19 2024 Epoch: 0
Epoch 0/500 0/34 loss: 0.9586 time 0.24s
Epoch 0/500 1/34 loss: 0.9595 time 0.10s
Epoch 0/500 2/34 loss: 0.9609 time 0.10s
Epoch 0/500 3/34 loss: 0.9598 time 0.10s
Epoch 0/500 4/34 loss: 0.9598 time 0.10s
Epoch 0/500 5/34 loss: 0.9588 time 0.10s
Epoch 0/500 6/34 loss: 0.9589 time 0.10s
Epoch 0/500 7/34 loss: 0.9599 time 0.10s
Epoch 0/500 8/34 loss: 0.9602 time 0.10s
Epoch 0/500 9/34 loss: 0.9600 time 0.10s
Epoch 0/500 10/34 loss: 0.9597 time 0.10s
Epoch 0/500 11/34 loss: 0.9600 time 0.10s
Epoch 0/500 12/34 loss: 0.9597 time 0.10s
Epoch 0/500 13/34 loss: 0.9600 time 0.10s
Epoch 0/500 14/34 loss: 0.9604 time 0.10s
Epoch 0/500 15/34 loss: 0.9606 time 0.10s
Epoch 0/500 16/34 loss: 0.9604 time 0.10s
Epoch 0/500 17/34 loss: 0.9598 time 0.10s
Epoch 0/500 18/34 loss: 0.9596 time 0.10s
Epoch 0/500 19/34 loss: 0.9599 time 0.10s
Epoch 0/500 20/34 loss: 0.9597 time 0.10s
Epoch 0/500 21/34 loss: 0.9599 time 0.10s
Epoch 0/500 22/3